In [1]:
%%capture
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
!unzip /content/ISIC2018_Task1-2_Training_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
!unzip /content/ISIC2018_Task1_Training_GroundTruth.zip
!pip install scikit-multilearn

In [10]:
from skmultilearn.adapt import MLkNN
import numpy as np
import os
import cv2

# Load Data

In [22]:
image_size = 128
path = "./"
train_path = "./ISIC2018_Task1-2_Training_Input"
epochs = 1
batch_size = 19

## Training Ids
train_ids = []
for files in os.walk(train_path):
   for name in files:
      train_ids.append(name)

train_ids = train_ids[2]
train_ids = [i.split('.', 1)[0] for i in train_ids]
val_data_size = 171

valid_ids = train_ids[:val_data_size]
train_ids = train_ids[val_data_size:]

smalltrain = train_ids[:10]
smalltest = train_ids[10:15]

trainx = []
trainy = []
testx = []
testy = []
for id in smalltrain:
    im = cv2.imread('./ISIC2018_Task1-2_Training_Input/' + id + '.jpg', 0)
    mask = cv2.imread('./ISIC2018_Task1_Training_GroundTruth/' + id + '_segmentation.png', 0)
    trainx.append(im)
    trainy.append(mask)
for id in smalltest:
    im = cv2.imread('./ISIC2018_Task1-2_Training_Input/' + id + '.jpg', 0)
    mask = cv2.imread('./ISIC2018_Task1_Training_GroundTruth/' + id + '_segmentation.png', 0)
    testx.append(im)
    testy.append(mask)

trainx = np.array(trainx)
trainy = np.array(trainy)
testx = np.array(testx)
testy = np.array(testy)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/lo

# Use MLkNN for Images/Segmentations - SEE ERROR

In [16]:
from scipy.sparse import csr_matrix

In [26]:
# ERROR

trainx = np.array([[255, 34, 2, 89], [233, 40, 94, 12]])
trainy =  np.array([[1,0,1,0], [1,1,0,1]])
k = int(np.log2(len(trainx)))
trainx = csr_matrix(trainx)
trainy = csr_matrix(trainy)
trainx, trainy

(<2x4 sparse matrix of type '<class 'numpy.longlong'>'
 	with 8 stored elements in Compressed Sparse Row format>,
 <2x4 sparse matrix of type '<class 'numpy.longlong'>'
 	with 5 stored elements in Compressed Sparse Row format>)

In [21]:
print('# of samples for trainx = ', trainx.shape)
print('# of samples for trainy = ', trainy.shape)
print('image dimension = ', trainx[0].shape)
print('label dimension = ', trainy[0].shape)
mlknn = MLkNN(k)
mlknn.fit(trainx, trainy)

# of samples for trainx =  (2, 8)
# of samples for trainy =  (2, 8)
image dimension =  (1, 8)
label dimension =  (1, 8)


TypeError: ignored

In [ ]:
num_classes = len(np.unique(trainy))
missing_label_indices = []
classes = np.array([0,1])
if classes.size != 0 and num_classes < len(classes):
    for idx, label in enumerate(classes):
        if label not in np.unique(trainy):
            missing_label_indices.append(idx)

In [ ]:
def predict_proba(testx):
    votes_per_example = mlknn.predict_proba(testx)
    if len(missing_label_indices) > 0:
        for i in missing_label_indices:
            new_col = np.zeros(votes_per_example.shape[0])
            votes_per_example = np.insert(votes_per_example, i, new_col, axis=1)
    return votes_per_example

In [ ]:
print(classes[np.argmax(predict_proba(testx), axis=3)])